# **Advanced RAG Technique and Evaluation**

In this document we will try the standard RAG versus the compressor based RAG.
We use GPT3.5 Turbo as LLM and will use as retriever a contextual compressor, which only takes the most relevant information from the retrieved documents by the similarity search.

Requirements: Please make sure to execute first LangChainRAG/Embedding-OpenAI-Chroma.ipynb to embed our medical documents. This Notebook is merely applying GPT3.5 Turbo as LLM.

In [49]:
!pip -q install langchain openai chromadb sentence_transformers evaluate rouge_score bert_score bleu_score

     ---------------------------------------- 61.1/61.1 kB 3.2 MB/s eta 0:00:00


ERROR: Could not find a version that satisfies the requirement bleu_score (from versions: none)
ERROR: No matching distribution found for bleu_score


In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

## **OpenAI Authenticatation**
We use OpenAIs GPT3.5 Turbo. Make sure to have balance on your OpenAI Dashboard and create a personal secret key at https://platform.openai.com/api-keys.

In [8]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


## **Load Chroma and GPT3.5 Turbo LLM**
We first load the Chroma vector database.

In [15]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings,GPT4AllEmbeddings,HuggingFaceBgeEmbeddings

In [4]:
import os

persist_directory = "./Chroma/chroma_openai"
# Create the directory if it does not exist
if not os.path.exists(persist_directory):
    print(f"Please execute first LangChainRAG/Embedding-OpenAI-Chroma.ipynb, we didn't find any Chroma vector storage.")
else:
    print(f"Directory '{persist_directory}' exists, perfect!")

Directory './Chroma/chroma_openai' exists, perfect!


In [10]:
db3 = Chroma(persist_directory=persist_directory, embedding_function=OpenAIEmbeddings())

In [20]:
from langchain import hub
from langchain_openai import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


retriever = db3.as_retriever() # print(dir(db3)) to get all functions, attributes
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [21]:
rag_chain.invoke("Why is the compatibility of drugs with excipients important in pharmaceutical formulations? And how can machine learning aid exactly?")

'The compatibility of drugs with excipients is crucial in pharmaceutical formulations to ensure properties like bioavailability and targeted delivery are optimized. Machine learning can aid in drug formulation development by providing predictive models that streamline the process, reducing the need for resource-intensive trial-and-error methods. Deep learning methods, such as convolutional neural networks, excel in analyzing complex datasets and can assist in optimizing drug formulations for improved efficacy.'

## **Load Pubmed QA Dataset**
We will now use the pubmed_qa from Hugging Face, which is merely a dataset that consists of many QA pairs. Each question has a detailed long answer. The idea is, that the answers retrieved by our LLMs should be "similar" to the answers from the dataset. For our purposes we only extract a random subset of the pairs.

In [25]:
from datasets import load_dataset,DatasetDict
dataset = load_dataset("pubmed_qa", "pqa_artificial")
num_test_samples = 100  # Choose the number of samples for the test set
# Assuming `dataset` is a DatasetDict and 'train' is the key for the training set
training_set = dataset['train']

# Create a test set by taking a subset of samples from the training set
test_set = training_set.shuffle(seed=42).select([i for i in range(num_test_samples)])
# Remove the selected samples from the training set, to avoid overlap
selected_pubids = [sample['pubid'] for sample in test_set]
training_set = training_set.filter(lambda x: x['pubid'] not in selected_pubids)

new_dataset_dict = DatasetDict({
    'train': training_set,
    'test': test_set,
})

dataset = new_dataset_dict
print(dataset)

questions=dataset["test"]["question"]
answers=dataset["test"]["long_answer"]

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 211169
    })
    test: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 100
    })
})


## **Generate answers using our original RAG.**
We now use rawly our GPT3.5 Turbo model to retrieve the answers for the questions.

In [26]:
from tqdm import tqdm

simple_answers = []
# Assuming 'questions' is your corpus of questions
for question in tqdm(questions, desc="Processing questions"):
    answer = rag_chain.invoke(question)
    simple_answers.append(answer)

Processing questions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:02<00:00,  1.22s/it]


## **Generate answers using contextual compression**
Here we use LLMChainExtractor to only take the relevant information from each document. We prepare the compressor based retriever and generate answers in an analogous way as above.

In [31]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor,LLMChainFilter
from langchain.llms import OpenAI

compressor = LLMChainExtractor.from_llm(
    llm=llm
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [36]:
print("Fist question in the set:",questions[10])
compressed_docs = compression_retriever.get_relevant_documents(questions[2])
compressed_docs

Fist question in the set: Does integrative analysis of methylome and transcriptome reveal the importance of unmethylated CpGs in non-CpG island gene activation?


C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[Document(page_content='telbivudine plus pegylated interferon alfa-2a in a randomized study in chronic hepatitis B', metadata={'keywords': '', 'seq_num': 76446, 'source/title': 'efficacy and safety of telbivudine treatment for the prevention of hbv perinatal transmission.'})]

In [37]:
rag_chain_compressor = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [38]:
compressor_answers = []
# Assuming 'questions' is your corpus of questions
for question in tqdm(questions, desc="Processing questions"):
    answer = rag_chain_compressor.invoke(question)
    compressor_answers.append(answer)

Processing questions:   0%|                                                                                                                                                               | 0/100 [00:00<?, ?it/s]C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.

Processing questions:  16%|████████████████████████                                                                                                                              | 16/100 [01:31<06:00,  4.30s/it]C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.

Processing questions:  32%|████████████████████████████████████████████████                                                                                                      | 32/100 [02:44<04:33,  4.02s/it]C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.

Processing questions:  48%|████████████████████████████████████████████████████████████████████████                                                                              | 48/100 [03:43<03:09,  3.65s/it]C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.

Processing questions:  64%|████████████████████████████████████████████████████████████████████████████████████████████████                                                      | 64/100 [14:44<08:26, 14.06s/it]C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.

Processing questions:  80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                              | 80/100 [15:40<01:09,  3.48s/it]C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.

Processing questions:  96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████      | 96/100 [16:42<00:14,  3.72s/it]C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
C:\Users\burha\miniconda3\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.

## **Evaluation**
We now use different metric scores to compare the performance of standard versus compressor based RAG.

In [55]:
import evaluate


bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

# Compute BLEU score
bleu_simple = evaluate.load("bleu").compute(predictions=simple_answers, references=answers)

# Compute ROUGE score
rouge_simple = evaluate.load("rouge").compute(predictions=simple_answers, references=answers)



# Compute BLEU score
bleu_compressor = evaluate.load("bleu").compute(predictions=compressor_answers, references=answers)

# Compute ROUGE score
rouge_compressor = evaluate.load("rouge").compute(predictions=compressor_answers, references=answers)


print("BLEU Score Simple:", bleu_simple)
print("BLEU Score Compressor:", bleu_compressor)
print("_________________________")
print("ROUGE Score Simple:", rouge_simple)
print("ROUGE Score Compressor:", rouge_compressor)

BLEU Score Simple: {'bleu': 0.01180448588384374, 'precisions': [0.22888616891064872, 0.05215123859191656, 0.024407252440725245, 0.012743628185907047], 'brevity_penalty': 0.2689199591546176, 'length_ratio': 0.43227513227513226, 'translation_length': 1634, 'reference_length': 3780}
BLEU Score Compressor: {'bleu': 0.07023031534837205, 'precisions': [0.261527201756526, 0.08127031757939485, 0.044370351372146705, 0.025796262174256384], 'brevity_penalty': 1.0, 'length_ratio': 1.0843915343915345, 'translation_length': 4099, 'reference_length': 3780}
_________________________
ROUGE Score Simple: {'rouge1': 0.06768921510771063, 'rouge2': 0.019146273536217943, 'rougeL': 0.05074483452402901, 'rougeLsum': 0.05142632571184533}
ROUGE Score Compressor: {'rouge1': 0.23949882750546148, 'rouge2': 0.08971561060992225, 'rougeL': 0.17811023084419322, 'rougeLsum': 0.178750720803475}


In [57]:
import numpy as np
### your code ###
bertscore_simple = bertscore.compute(predictions=simple_answers, references=answers, lang="en")
bertscore_compressor = bertscore.compute(predictions=compressor_answers, references=answers, lang="en")
bertscore_simple_averaged={}
bertscore_compressor_averaged={}
for key in bertscore_simple.keys():
  if key!='hashcode':
    bertscore_simple_averaged[key]=np.mean(bertscore_simple[key])
    bertscore_compressor_averaged[key]=np.mean(bertscore_compressor[key])

print("BERT Score Simple:",bertscore_simple_averaged)
print("BERT Score Compressor:",bertscore_compressor_averaged)

BERT Score Simple: {'precision': 0.8288946324586868, 'recall': 0.8197180956602097, 'f1': 0.8241216886043549}
BERT Score Compressor: {'precision': 0.86609958589077, 'recall': 0.8680021101236344, 'f1': 0.8668680435419083}


## **Conclusion:**

We can clearly see, that contextual compression improves the accuracy of the RAG.